In [4]:
import pandas as pd
data.to_csv("datosstockMSFT.csv")

In [2]:
import pandas as pd
data = pd.read_csv("datosETH.BTC5m.csv")

In [11]:
from math import floor
%matplotlib widget
import matplotlib.pyplot as plt
from pylab import rcParams
from statistics import mean
from IPython.display import clear_output
rcParams['figure.figsize'] = 10, 4
dinero=100000
portfolio=0
comisión=0.0000 #0.02% es lo normal
dinero_inicial=dinero
dinerototal=dinero_inicial
dataclose=list(data["close"])
#dataclose.reverse()
datavol=list(data["volume"])
#datavol.reverse()
import numpy as np
import tensorflow as tf
import keras

from keras.optimizers import SGD
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Conv1D
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers.merge import concatenate

close_input = keras.Input(shape=(None,), name="close")
volume_input = keras.Input(shape=(None,), name="volume")

close_features = tf.expand_dims(close_input, axis=-1)
volume_features = tf.expand_dims(volume_input, axis=-1)


close_features = layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(close_features)
close_features = layers.Dropout(0.5)(close_features)


volume_features = layers.Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(volume_features)
volume_features = layers.Dropout(0.5)(volume_features)


close_features = layers.LSTM(32)(close_features)
close_features = layers.Dropout(0.2)(close_features)


volume_features = layers.LSTM(32)(volume_features)
volume_features = layers.Dropout(0.2)(volume_features)

x = concatenate([close_features,volume_features])
#x=Flatten()(x)
x = layers.Dense(128)(x)
x = layers.Dense(128)(x)

output = layers.Dense(1,activation="sigmoid",name="futuro")(x)
#output = layers.Dense(1,activation="softmax",name="futuro")(x)
model = keras.Model(inputs=[close_input,volume_input],outputs=[output])

keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

model.compile(optimizer="adagrad", loss='binary_crossentropy',metrics=["accuracy"])
model.load_weights("pruebaGOOGLE100EPOCH.h5")


def brain(realdata,realdatavol,cutshort,cutlong):

    realdatarel=[]
    realdatarelvol=[]
    
    for i in range(1,len(realdata)):
        realdatarel.append(((realdata[i]/realdata[i-1])-1)*100)
    for i in range(10,len(realdatavol)):
        realdatarelvol.append(((realdatavol[i]/mean(realdatavol[i-5:i-1]))-1))
    
    realdatarel_array = np.array(realdatarel)
    ncols = realdatarel_array.shape[0]
    realdatarel_array = realdatarel_array.reshape(1, ncols, 1)
    
    realdatarelvol_array = np.array(realdatarelvol)
    ncols = realdatarelvol_array.shape[0]
    realdatarelvol_array = realdatarelvol_array.reshape(1, ncols, 1)
    #modelo red

    #predecir y aplicar el cut para el return
    realdata_Y = model.predict({"close":realdatarel_array,"volume":realdatarelvol_array})

    if realdata_Y < cutshort:
        return -1 #vender
    elif realdata_Y > cutlong:
        return 1 #comprar long
    else:
        return 0 #no está seguro

def comprar(acciones,precio):
    global dinero,portfolio
    if dinero-acciones * precio < 0:
        return False
    dinero = dinero-(acciones * precio)*(1+comisión)
    portfolio = portfolio + acciones
    return True

def vender (acciones, precio):
    global dinero,portfolio
    if portfolio - acciones < 0:
        return False
    dinero = dinero+(acciones * precio)*(1-comisión)
    portfolio = portfolio - acciones
    return True

def comprar_porcentaje(porcentaje,precio): #Gasta un porcentaje de tu dinero en acciones
    global dinero,portfolio
    if (dinero*porcentaje)/precio==0:
        return False
    acciones = (dinero*porcentaje)/precio
    comprar(acciones,precio)
    return True

def vender_porcentaje(porcentaje,precio): #Vende un porcentaje de tu dinero total
    global dinero,portfolio
    if portfolio*porcentaje==0:
        return False
    acciones= portfolio*porcentaje
    vender(acciones,precio)
    return True

class Historial:
    def __init__(self):
        self.dinerototal=[]
        self.dinero=[]
        self.precio=[]
        self.portfolio=[]
        self.decisiones=[]
    def savetimestep(self,dinero,portfolio,dinerototal,precio,decisiones):
        self.dinero.append(dinero)
        self.portfolio.append(portfolio)
        self.dinerototal.append(dinerototal)
        self.precio.append(precio)
        self.decisiones.append(decisiones)
    def dibujar(self,arg="dinerototal"):
        global dinero_inicial
        if arg == "precio":
            print("Precio")

            plt.plot(self.precio)
            
            for i in range(len(self.decisiones)):
                if self.decisiones[i] == 1:
                    plt.scatter(i,self.precio[i],c="green",marker="^")
                if self.decisiones[i] == 0:
                    continue
                if self.decisiones[i] == -1:
                    plt.scatter(i,self.precio[i],c="red",marker="v")
            plt.show()
        if arg == "portfolio":
            print("Portfolio")

            plt.plot(self.portfolio)
            for i in range(len(self.decisiones)):
                if self.decisiones[i] == 1:
                    plt.scatter(i,self.portfolio[i],c="green",marker="^")
                if self.decisiones[i] == 0:
                    continue
                if self.decisiones[i] == -1:
                    plt.scatter(i,self.portfolio[i],c="red",marker="v")

        if arg == "dinerototal":
            #print("Dinero total")

            plt.plot(self.dinerototal, color="blue",label="Estrategia IA")
            
            hold = [i * (dinero_inicial/self.precio[0]) for i in self.precio]
            plt.plot(hold,color="magenta",label="Estrategia hold")
            
            for i in range(len(self.decisiones)):
                if self.decisiones[i] == 1:
                    plt.scatter(i,self.dinerototal[i],c="green",marker="^")
                if self.decisiones[i] == 0:
                    continue
                if self.decisiones[i] == -1:
                    plt.scatter(i,self.dinerototal[i],c="red",marker="v")
            plt.show()

historial = Historial()
timesteps=96000
decisiones=0
action=False
for timestep in range(0,timesteps):
    if timestep==timesteps-1:
        clear_output(wait=True)
        print(("■"*10)+"   "+str(timesteps)+"/"+str(timesteps), flush=True)
    elif timestep%(timesteps*0.01)==0:
        clear_output(wait=True)
        print("■"*int(((timestep*10)//timesteps))+"□"*int((10-(((timestep*10))//timesteps)))+"   "+str(timestep)+"/"+str(timesteps)+"     Dinero total: "+str(str("{:.2f}".format(round(dinerototal,2)))), flush=True)
    if timestep < 100:
        continue
    precio=dataclose[timestep]
    ultimosprecios=dataclose[timestep-10:timestep+1]
    volumen=datavol[timestep]
    ultimosvolumenes=datavol[timestep-10:timestep+1]
    #algoritmo compra/venta

    if action==True:
        if decisiones==1 and float(precio)/mean(dataclose[timestep-2:timestep+1])>1:
            vender_porcentaje(1,precio)
            
            dinerototal=dinero+precio*portfolio
            historial.savetimestep(dinero,portfolio,dinerototal,precio,-1)
            decisiones=0
            action=False
            #continue
        elif decisiones==-1 and float(precio)/mean(dataclose[timestep-2:timestep+1])<1:
            vender_porcentaje(1,precio)
            decisiones=1
            dinerototal=dinero+precio*portfolio
            historial.savetimestep(dinero,portfolio,dinerototal,precio,decisiones)
            decisiones=0
            action=False
            #continue
        else:

            dinerototal=dinero+precio*portfolio
            historial.savetimestep(dinero,portfolio,dinerototal,precio,0)
            continue
 #0.46,0.49,0.5,0.5
    decisiones=brain(ultimosprecios,ultimosvolumenes,0.435,0.535)
    if decisiones ==1 and float(precio)/mean(dataclose[timestep-2:timestep+1])<1:
        comprar_porcentaje(1,precio)
        action=True
    elif decisiones==-1 and float(precio)/mean(dataclose[timestep-2:timestep+1])>1:
        comprar_porcentaje(1,precio)
        action=True
    #fin algoritmo compra/venta
    dinerototal=dinero+precio*portfolio
    historial.savetimestep(dinero,portfolio,dinerototal,precio,decisiones)
print("Dinero total: "+str("{:.2f}".format(round(int(dinerototal),2))))
print("Beneficio: "+str("{:.2f}".format(((dinerototal/dinero_inicial)-1)*100))+"%")
historial.dibujar("dinerototal")
#historial.dibujar("precio")

■■■■■■■■■■   96000/96000
Dinero total: 199603.00
Beneficio: 99.60%


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
data

,time,low,high,open,close,volume
0,2020-02-07 00:05:00,0.02186,0.02189,0.02187,0.02188,15.922641
1,2020-02-07 00:10:00,0.02191,0.02194,0.02191,0.02192,3.300343
2,2020-02-07 00:15:00,0.02191,0.02195,0.02193,0.02194,88.327171
3,2020-02-07 00:20:00,0.02193,0.02198,0.02195,0.02198,50.471752
4,2020-02-07 00:25:00,0.02198,0.02206,0.02198,0.02203,457.668288
...,...,...,...,...,...,...
105132,2021-02-07 01:25:00,0.04247,0.04260,0.04258,0.04254,111.529175
105133,2021-02-07 01:30:00,0.04252,0.04260,0.04256,0.04253,73.206711
105134,2021-02-07 01:35:00,0.04243,0.04257,0.04255,0.04244,105.455067
105135,2021-02-07 01:40:00,0.04227,0.04251,0.04245,0.04229,137.833174


In [66]:
print("Beneficio: "+str("{:.2f}".format(((dinerototal/100000)-1)*100))+"%")

Beneficio: 10.87%


In [78]:
print("Dinero total: "+str("{:.2f}".format(round(int(dinerototal),2))))

Dinero total: 100366.00


In [39]:
data

,time,low,high,open,close,volume
0,2019-02-07 01:00:00,0.03069,0.03096,0.03069,0.03089,418.231200
1,2019-02-07 02:00:00,0.03077,0.03090,0.03090,0.03079,267.197109
2,2019-02-07 03:00:00,0.03078,0.03080,0.03080,0.03080,298.578935
3,2019-02-07 04:00:00,0.03076,0.03082,0.03080,0.03077,1055.214151
4,2019-02-07 05:00:00,0.03073,0.03079,0.03077,0.03076,89.141402
...,...,...,...,...,...,...
17534,2021-02-06 20:00:00,0.04219,0.04276,0.04253,0.04224,5224.612098
17535,2021-02-06 21:00:00,0.04214,0.04275,0.04225,0.04268,2214.852597
17536,2021-02-06 22:00:00,0.04231,0.04280,0.04273,0.04257,1968.588789
17537,2021-02-06 23:00:00,0.04256,0.04300,0.04257,0.04278,2425.260704


In [21]:
dataclose[16900:17000]

[0.03143,
 0.03121,
 0.031330000000000004,
 0.031219999999999998,
 0.03119,
 0.030930000000000003,
 0.03035,
 0.030389999999999997,
 0.030539999999999998,
 0.03052,
 0.03052,
 0.030389999999999997,
 0.030780000000000002,
 0.03069,
 0.030389999999999997,
 0.030789999999999998,
 0.031089999999999996,
 0.03113,
 0.03108,
 0.03149,
 0.03161,
 0.03128,
 0.0312,
 0.031039999999999998,
 0.0309,
 0.03105,
 0.031030000000000002,
 0.03082,
 0.031,
 0.031160000000000004,
 0.03162,
 0.031939999999999996,
 0.03152,
 0.03123,
 0.03151,
 0.03106,
 0.03119,
 0.03085,
 0.03048,
 0.031110000000000002,
 0.0312,
 0.03094,
 0.03084,
 0.03112,
 0.030930000000000003,
 0.030760000000000003,
 0.03069,
 0.03088,
 0.030930000000000003,
 0.030860000000000002,
 0.030869999999999998,
 0.030869999999999998,
 0.03083,
 0.030889999999999997,
 0.030860000000000002,
 0.030869999999999998,
 0.030910000000000003,
 0.030610000000000002,
 0.03067,
 0.03023,
 0.03005,
 0.03022,
 0.03003,
 0.029789999999999997,
 0.02975,
 0.0